In [12]:
import pandas as pd, numpy as np 
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
labels = train.columns[2:]
ys_valid = valid[labels]

## COMBINE TOXIC CATEGORIES
y_valid = ys_valid.sum(axis=1)
y_valid.loc[y_valid>1] = 1

In [3]:
## Load SVM, XGBoost, and GRU results
NBSVM = pd.read_csv('../artifacts/simple/preds/tfidf_nbsvm.csv')
LSTM = pd.read_csv('../artifacts/simple/preds/glove_lstm.csv')

In [10]:
preds = (LSTM['glove_gru'] + NBSVM['tfidf_nbsvm'])/2
preds[preds<=0.5] = 0
preds[preds>0.5] = 1

In [14]:
# Evaluate predictions
results = pd.DataFrame(columns=['Label','Accuracy', 'Recall', 'Precision', 'F1', 'Vectorizer', 'model'])
acc, prec, recall, f1 = (accuracy_score(y_valid, preds), 
                            precision_score(y_valid, preds), 
                            recall_score(y_valid, preds), 
                            f1_score(y_valid, preds))

results = results.append({'Label': 'Toxic_Combined',
                        'Accuracy':acc,
                        'Recall':recall,
                        'Precision':prec,
                        'F1':f1,
                        'Vectorizer':'tfidf',
                        'model': 'NB_SVM'}, 
                        ignore_index = True)

print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                'Toxic_combined', 
                                acc, 
                                prec, 
                                recall,
                                f1))

Results for Toxic_combined comments: Accuracy - 0.97; Precision - 0.88; Recall - 0.78; F1 - 0.83
